# Gravitational Models Comparison

In this tutorial, we'll compare various **Gravitational Models**.

## Setup

Let's import the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.time import Instant
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel

## Computation

In [ ]:
environment = Environment.default()
earth = environment.access_celestial_object_with_name("Earth")

Let's set up some gravitational models:

In [ ]:
gravitational_models = [
    EarthGravitationalModel(EarthGravitationalModel.Type.Spherical),
    EarthGravitationalModel(EarthGravitationalModel.Type.WGS84),
    EarthGravitationalModel(EarthGravitationalModel.Type.EGM84),
    EarthGravitationalModel(EarthGravitationalModel.Type.EGM96),
]

And evaluate them at various altitudes:

In [ ]:
altitudes = list(range(0, 1000000, 100000))

In [ ]:
instant = Instant.J2000()

In [ ]:
positions = [
    np.array([float(earth.get_equatorial_radius().in_meters() + altitude), 0.0, 0.0])
    for altitude in altitudes
]

In [ ]:
def evaluate_field_values(gravitational_model, positions):
    field_values = [
        np.transpose(gravitational_model.get_field_value_at(position, instant))[0]
        for position in positions
    ]

    return field_values

In [ ]:
field_valuess = [
    evaluate_field_values(gravitational_model, positions)
    for gravitational_model in gravitational_models
]
print(field_valuess)
print(altitudes)

In [ ]:
field_dfs = [
    pd.DataFrame(
        [
            [altitude, field.tolist()]
            for [altitude, field] in zip(altitudes, field_values)
        ],
        columns=["$altitude [m]$", "$g_x$"],
    )
    for field_values in field_valuess
]

# field_dfs = [pd.DataFrame([[altitude, field.tolist()] for [altitude, field] in zip(altitudes, field_values)], columns=['$altitude [m]$', '$g_x$', '$g_y$', '$g_z$']) for field_values in field_valuess]

## Display

Table:

In [ ]:
field_dfs[0]

Scatter plot:

In [ ]:
figure = go.Figure(
    data=[
        go.Scatter(x=field_df["$altitude [m]$"], y=field_df["$g_x$"])
        for field_df in field_dfs
    ]
)

figure.show()

---